In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
import visualizer_tutorial as vistut

vistut.quiet()  # turn off default logging and most warnings
from idaes.core.util.model_statistics import degrees_of_freedom
from IPython.display import Markdown

In [3]:
# use the pre-defined function to create the flowsheet
model = vistut.create_model()

# description of the flowsheet we created
display(Markdown(vistut.function_markdown(vistut.create_model)))

vistut.quiet()

# initialize the flowsheet as a square problem (dof=0)
vistut.initialize_model(model)

# verify that there are zero degrees of freedom
print(f"DOF = {degrees_of_freedom(model)}")

## Create an IDAES flowsheet for hydrodealkylation (HDA)
### About HDA

Hydrodealkylation (HDA) is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**

This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will not cover in this example.

### References

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.


DOF = 0


In [4]:
# Add a second flash unit
from idaes.models.unit_models import Flash
from pyomo.network import Arc
from pyomo.environ import Expression, TransformationFactory

m = model  # alias
m.fs.F102 = Flash(
    property_package=m.fs.thermo_params,
    has_heat_transfer=True,
    has_pressure_change=True,
)
# connect to 1st flash unit
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)
# update expressions for purity and cost
m.fs.purity = Expression(
    expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
    / (
        m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
        + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]
    )
)
m.fs.heating_cost = Expression(
    expr=2.2e-7 * m.fs.H101.heat_duty[0] + 1.9e-7 * m.fs.F102.heat_duty[0]
)
# fix unit output and pressure drop
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

# expand arcs
TransformationFactory("network.expand_arcs").apply_to(m)
# re-initialize
_ = vistut.initialize_model(m)

In [5]:
# v model.fs.visualize("Hydrodealkylation-new")

In [6]:
# Create the solver object
from pyomo.environ import SolverFactory

solver = SolverFactory("ipopt")
solver.options = {"tol": 1e-6, "max_iter": 5000}

# Solve the model
results = solver.solve(model, tee=False)

# Open a second window
model.fs.visualize("HDA_solved")

Started visualization server
Loading saved flowsheet from 'HDA_solved.json'
Saving flowsheet to default file 'HDA_solved.json' in current directory (C:\Users\dang\src\idaes\dangunter\idaes-examples\idaes_examples\notebooks\docs\tut)
_build_abstract_layout - node_name: hydrogen_feed_1
_build_abstract_layout - node_name: toluene_feed_1
_build_abstract_layout - node_name: M101
_build_abstract_layout - node_name: H101
_build_abstract_layout - node_name: R101
_build_abstract_layout - node_name: F101
_build_abstract_layout - node_name: S101
_build_abstract_layout - node_name: F102
_build_abstract_layout - node_name: C101
_build_abstract_layout - node_name: vap_outlet_1
_build_abstract_layout - node_name: liq_outlet_1


Flowsheet visualization at: http://localhost:49310/app?id=HDA_solved
